In [21]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import random
import os
from torchvision import datasets
import torch.utils.data as data
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score
from PIL import Image
from rdkit import Chem
from rdkit.Chem import AllChem

df_data = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/level5_beta_trt_cp_24h_10uM.csv')
use_HQ_sample_id = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/revise_use_LINCS_HQ_data_pretrain_sample_id.csv')['x']
df_res = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
cellline_map = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/CellLineEncode/use_training_cell_line_expression_features_128_encoded_20240111.csv')
cell_list = cellline_map['Unnamed: 0'].to_list()
df_res = df_res[df_res['cell_iname'].isin(cell_list)]

print(len(df_res))
pool_drug = set(df_res.SMILES.to_list())

for cell in ['MCF7', 'PC3', 'A549']:
        use_HQ_sample_id = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/revise_use_LINCS_HQ_data_target_cellline_{cell}_sample_id.csv')['x']
        # tmp = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/GPS_predictable_genes.csv')
        # genelist = tmp.x.to_list()
        
        
        df_target = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
        # median = df_target[['SMILES']+genelist].groupby(by='SMILES').median().reset_index()
        # median['cellline'] = cell
        # df_target = median
        df_target = df_target.rename(columns={'SMILES': 'smiles'})
        # for g in genelist:
        #     df_target[g] = df_target[g].apply(lambda x: (x > 1.5) * 1 + (x >= -1.5) * 1)

        Drugs = df_target['smiles'].to_list()
        tmp = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/{cell}_internal_val2_data.csv')
        valDrugs = tmp.SMILES.to_list()
        trainDrugs = set(Drugs)-set(valDrugs)
        print(len(Drugs), len(list(trainDrugs)), len(valDrugs))
        pool_drug = pool_drug.intersection(trainDrugs)
        print(len(list(pool_drug)))

        # df_train = df_target[df_target['smiles'].isin(trainDrugs)]
        # df_val = df_target[df_target['smiles'].isin(valDrugs)]


7258
2084 1203 134
810
2164 1272 141
497
1562 842 94
279


In [22]:
import pickle
with open('HQ_pool_drug.pkl', 'wb') as f:
    pickle.dump(list(pool_drug), f)

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import random
import os, pickle
from torchvision import datasets
import torch.utils.data as data
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score
from PIL import Image
from rdkit import Chem
from rdkit.Chem import AllChem

In [3]:
a = torch.rand(9,10)
a

tensor([[0.2612, 0.2648, 0.5329, 0.2044, 0.5449, 0.6778, 0.7799, 0.7144, 0.5464,
         0.4501],
        [0.3396, 0.1379, 0.6470, 0.7493, 0.2567, 0.7192, 0.5119, 0.4172, 0.2098,
         0.0164],
        [0.1395, 0.3654, 0.4532, 0.7445, 0.6429, 0.1704, 0.5869, 0.3531, 0.9040,
         0.6901],
        [0.3571, 0.3114, 0.3720, 0.7654, 0.5273, 0.2036, 0.0676, 0.8054, 0.2283,
         0.2747],
        [0.1931, 0.6388, 0.8729, 0.1342, 0.1833, 0.4717, 0.3625, 0.7046, 0.0659,
         0.4700],
        [0.3470, 0.1281, 0.3182, 0.6841, 0.4964, 0.1919, 0.8253, 0.8289, 0.5368,
         0.6650],
        [0.1000, 0.2683, 0.5350, 0.0772, 0.3491, 0.8917, 0.6664, 0.4180, 0.0076,
         0.3310],
        [0.3566, 0.1236, 0.9572, 0.5597, 0.4608, 0.3390, 0.7819, 0.9701, 0.1375,
         0.0689],
        [0.9911, 0.5844, 0.7553, 0.9817, 0.6434, 0.9404, 0.9557, 0.1824, 0.6003,
         0.9934]])

In [5]:
torch.unsqueeze(a.mean(0),0)

tensor([[0.3428, 0.3136, 0.6049, 0.5445, 0.4561, 0.5117, 0.6154, 0.5993, 0.3596,
         0.4399]])

In [10]:
torch.concatenate((a,b),0)

tensor([0.3964, 0.8496, 0.9765, 0.1756, 0.2676, 0.2418, 0.4815, 0.9166, 0.5119,
        0.1503, 0.1443, 0.1001, 0.7114, 0.8827])